In [2]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/

import pandas as pd  
import numpy as np
import pandas_datareader.data as web
import datetime as dt
from datetime import date, datetime, timedelta

import scipy.optimize as sco
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#globals
equities = ['SPY', 'IWM', 'QQQ', 'XLK', 'XLE', 'XRT', 'XRT', 'XLU', 'XLF','XLI','XLV', 'XAR']
fixed_income = ['TLT', 'IEF']
precious_metals = ['GLD', 'GDX', 'SLV', 'GDXJ']

tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.02

In [3]:
#credits to https://www.pythonforfinance.net/2019/07/02/investment-portfolio-optimisation-with-python-revisited/

def get_benchmark(indices,start, end):
    indices = indices

    starter_data = {'benchmark':'0','ret':'0', 'stdev':'0', 'sharpe':'0'}
    df = pd.DataFrame(starter_data, index = starter_data.keys())[0:1].reset_index(drop = True)

    for index in indices:

        series = web.DataReader(index, 'yahoo', start, end)['Adj Close']
        ret = series.pct_change().mean()*252
        stdev = series.pct_change().std()*np.sqrt(252)
        sharpe = (ret-rf)/stdev
        benchmark_data = {'benchmark':index,'ret':ret, 'stdev':stdev, 'sharpe':sharpe}
        new_row = pd.DataFrame(benchmark_data, index = benchmark_data.keys())[0:1].reset_index(drop = True)
        df = df.append(new_row)
    df = df.reset_index(drop = True).drop(0)
    return(df)

def ticker_data(tickers, start, end):
    days = (end-start).days

    index = pd.date_range(start, periods = days, freq='D')
    df = pd.DataFrame(index = index,columns = tickers)

    for ticker in tickers:
        df[ticker] = web.DataReader(ticker, 'yahoo', start, end)['Adj Close']
    df = df.dropna()
    return(df)

def portfolio_data(portfolio, start, end, rf, leverage, margin_int):
    
    weights = np.array(list(portfolio.values()))
    tickers = list(portfolio.keys())
    data = ticker_data(tickers, start, end)
    mean_returns = data.pct_change().mean()
    cov = data.pct_change().cov()
    
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    
    levered_return = portfolio_return*leverage - (leverage - 1)*margin_int
    levered_std = portfolio_std * leverage
    levered_sharpe = (levered_return - rf) / levered_std

    portfolio_stats = {'ret': portfolio_return, 'stdev':portfolio_std, 'sharpe': sharpe_ratio,
                      'levered ret': levered_return, 'levered stdev':levered_std, 'levered sharpe': levered_sharpe}
    
    portfolio_df = pd.DataFrame(portfolio_stats,columns=['ret','stdev','sharpe'] +list(portfolio.keys())+ ['levered ret','levered stdev','levered sharpe'], index = [0])
    
    for ticker in tickers:
        portfolio_df[ticker] = portfolio[ticker]
    
    return(portfolio_df)

In [4]:
def calc_portfolio_perf(weights, mean_returns, cov, rf):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    return portfolio_return, portfolio_std, sharpe_ratio

def simulate_random_portfolios(num_portfolios, mean_returns, cov, rf):
    results_matrix = np.zeros((len(mean_returns)+3, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio = calc_portfolio_perf(weights, mean_returns, cov, rf)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+3,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe'] + [ticker for ticker in tickers])
        
    return results_df

#simulate leveraged portfolios

def calc_portfolio_perf_lev(weights, mean_returns, cov, rf, leverage, margin_int):
    portfolio_return = np.sum(mean_returns * weights) * 252
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov, weights))) * np.sqrt(252)
    sharpe_ratio = (portfolio_return - rf) / portfolio_std
    
    levered_return = portfolio_return*leverage - (leverage - 1)*margin_int
    levered_std = portfolio_std * leverage
    levered_sharpe = (levered_return - rf) / levered_std
    
    return portfolio_return, portfolio_std, sharpe_ratio, levered_return, levered_std, levered_sharpe

def simulate_random_portfolios_lev(num_portfolios, mean_returns, cov, rf, leverage, margin_int):
    results_matrix = np.zeros((len(mean_returns)+6, num_portfolios))
    for i in range(num_portfolios):
        weights = np.random.random(len(mean_returns))
        weights /= np.sum(weights)
        portfolio_return, portfolio_std, sharpe_ratio, levered_return, levered_std, levered_sharpe = calc_portfolio_perf_lev(weights, mean_returns, cov, rf, leverage, margin_int)
        results_matrix[0,i] = portfolio_return
        results_matrix[1,i] = portfolio_std
        results_matrix[2,i] = sharpe_ratio
        results_matrix[3,i] = levered_return
        results_matrix[4,i] = levered_std
        results_matrix[5,i] = levered_sharpe
        
        #iterate through the weight vector and add data to results array
        for j in range(len(weights)):
            results_matrix[j+6,i] = weights[j]
            
    results_df = pd.DataFrame(results_matrix.T,columns=['ret','stdev','sharpe','levered ret','levered stdev','levered sharpe'] + [ticker for ticker in tickers])
        
    return results_df

In [5]:
#find target leverage

def apply_leverage(input_portfolio, leverage, margin_int):
    
    results_frame = input_portfolio.copy()
    results_frame['levered ret']=[0.0]*len(results_frame)
    results_frame['levered stdev']=[0.0]*len(results_frame)
    results_frame['levered sharpe'] = [0.0]*len(results_frame)
    
    for i in range(len(results_frame)):
        results_frame['levered ret'][i] = results_frame['ret'][i] * leverage - (leverage - 1)*margin_int
        results_frame['levered stdev'][i] = results_frame['stdev'][i] * leverage
        results_frame['levered sharpe'][i] = results_frame['levered ret'][i]/results_frame['levered stdev'][i]
        
    return(results_frame)

def target_risk(portfolios, acceptable_risk, start, end, margin_int):
    max_sharpe = portfolios['sharpe'].max()
    optimal_portfolio = portfolios[portfolios['sharpe']==max_sharpe]
    leverage = float(acceptable_risk / optimal_portfolio['stdev'])
    
    levered_portfolios = apply_leverage(portfolios, leverage, margin_int)
    optimal_levered_portfolio = levered_portfolios[levered_portfolios['sharpe']==max_sharpe]
    optimal_levered_portfolio['leverage'] = leverage
    
    return(optimal_levered_portfolio)

def target_return(portfolios, tgt_rtn, start, end, margin_int):
    max_sharpe = portfolios['sharpe'].max()
    optimal_portfolio = portfolios[portfolios['sharpe']==max_sharpe]
    
    leverage = (tgt_rtn - margin_int)/(float(optimal_portfolio['ret']-margin_int))
    
    levered_portfolios = apply_leverage(portfolios, leverage, margin_int)
    optimal_levered_portfolio = levered_portfolios[levered_portfolios['sharpe']==max_sharpe]
    optimal_levered_portfolio['leverage'] = leverage
    
    return(optimal_levered_portfolio)

def capital_allocation(NAV, input_results, tickers):
    pct_allocation = {}
    cap_allocation = {}
    for ticker in tickers:
        pct_allocation[ticker] = float(input_results[ticker])
        cap_allocation[ticker] = round(pct_allocation[ticker]*NAV,2)
    return(cap_allocation)

In [6]:
indices = ['SPY', 'IWM', 'VT', 'VEF.TO', 'VTI']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
standard = get_benchmark(indices,start, end)
standard

,benchmark,ret,stdev,sharpe
1,SPY,0.104362,0.199618,0.522807
2,IWM,0.0972809,0.248786,0.391022
3,VT,0.0850438,0.22475,0.378392
4,VEF.TO,0.0907691,0.168605,0.538353
5,VTI,0.105993,0.201052,0.527193


In [11]:
tickers = ['SPY', 'GLD', 'TLT', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
ticker_df = ticker_data(tickers, start, end)
ticker_df.head()

,SPY,GLD,TLT,LQD
2005-07-25,90.983475,42.509998,56.903557,59.293636
2005-07-26,91.094292,42.250000,56.995152,59.304337
2005-07-27,91.426628,42.439999,56.903557,59.229378
2005-07-28,92.002747,42.750000,57.410233,59.454315
2005-07-29,91.389702,42.820000,56.842503,59.320442


In [10]:
portfolio = {'TLT':0.402064, 'VTI':0.293876, 'GLD':0.157595, 'LQD':0.146465}
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
rf = 0.0
leverage = 1.5
margin_int = 0.015

portfolio_DF = portfolio_data(portfolio, start, end, rf, leverage, margin_int)
portfolio_DF

,ret,stdev,sharpe,TLT,VTI,GLD,LQD,levered ret,levered stdev,levered sharpe
0,0.090287,0.080004,1.128538,0.402064,0.293876,0.157595,0.146465,0.127931,0.120006,1.066041


In [11]:
tickers = ['TLT', 'VTI', 'GLD', 'LQD']
start = date.today() - timedelta(days = 3650*1.5)
end = date.today()
df = ticker_data(tickers,start,end)

mean_returns = df.pct_change().mean()
cov = df.pct_change().cov()
num_portfolios = 10000
rf = 0.0
results_frame = simulate_random_portfolios(num_portfolios, mean_returns, cov, rf)
results_frame.head()

,ret,stdev,sharpe,TLT,VTI,GLD,LQD
0,0.091018,0.082149,1.107964,0.440205,0.356134,0.097563,0.106097
1,0.093621,0.104873,0.892704,0.307703,0.082228,0.462420,0.147649
2,0.085246,0.077077,1.105988,0.260556,0.212274,0.195473,0.331697
3,0.096555,0.089533,1.078424,0.323606,0.346180,0.270072,0.060142
4,0.092937,0.099339,0.935554,0.060566,0.357510,0.305086,0.276838


In [14]:
max_sharpe = results_frame['sharpe'].max()
results_frame[results_frame['sharpe']==max_sharpe]


,ret,stdev,sharpe,TLT,VTI,GLD,LQD
5810,0.09039,0.08008,1.128749,0.396332,0.295928,0.160329,0.147411


In [476]:
standard

,benchmark,ret,stdev,sharpe
1,SPY,0.104145,0.199594,0.521784
2,IWM,0.0968185,0.24876,0.389205
3,VT,0.0850438,0.22475,0.378392
4,QQQ,0.157312,0.214189,0.734452


In [ ]:
def stock_correlation(stock1, stock2):
    correlation = stock1['Close'].pct_change().corr(stock2['Close'].pct_change())
    return(correlation)

def get_correlation(stocks, start, end):
    
    assets ={}
    assets2 = {}

    for stock in stocks:
        assets[stock] = ticker_df(stock, start, end)

    for stock in stocks:
        assets2[stock] = list(assets[stock]['Close'])

    df = pd.DataFrame(data = assets2, columns = assets.keys())
    corr_matrix = df.corr()

    for stock1 in stocks:
        for stock2 in stocks:
            corr_matrix[stock1][stock2] = stock_correlation(assets[stock1], assets[stock2])

In [66]:
def series_corr(ser1, ser2):
    ser1pct = ser1.pct_change()
    ser2pct = ser2.pct_change()
    correlation = ser1pct.corr(ser2pct)
    return(correlation)

def create_corr_matrix(ticker_df):

    corr_matrix = ticker_df.corr()

    for stock1 in list(ticker_df.columns):
        for stock2 in list(ticker_df.columns):
            #corr_matrix[stock1][stock2] = series_corr(ticker_df[stock1], ticker_df[stock2])
            try:
                stock1_ser = ticker_df[stock1].pct_change() 
                stock2_ser = ticker_df[stock2].pct_change()
                corr_matrix[stock1][stock2] = stock1_ser.corr(stock2_ser)
                
            except:
                pass
    return(corr_matrix)

In [ ]:
for stock1 in ticker_df.columns:
    print(

In [68]:
tickers = ['SPY', 'IWM', 'QQQ', 'GDX', 'GDXJ', 'GLD', 'SLV', 'USO', 'XLK', 'XLE', 'XRT', 'XRT', 'XLU', 'XLF','XLI','XLV', 'XAR', 'ARKK', 'TLT', 'LQD', 'IEF']
#start = date.today() - timedelta(days = 3650*1.5)
#end = date.today()

start = dt.datetime(2007,1,1)
end = dt.datetime(2020,6,30)


ticker_df = ticker_data(tickers, start, end)
#ticker_df.head()

In [42]:
ser1 = ticker_df['SPY'].pct_change()
ser2 = ticker_df['QQQ'].pct_change()
ser1.corr(ser2)

0.9348735882585

In [46]:
ser1 = ticker_df['SPY']
ser2 = ticker_df['QQQ']
series_corr(ser1, ser2)

0.9348735882585

In [69]:
create_corr_matrix(ticker_df)

c:\users\olfoa\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,SPY,IWM,QQQ,GDX,GDXJ,GLD,SLV,USO,XLK,XLE,...,XRT,XLU,XLF,XLI,XLV,XAR,ARKK,TLT,LQD,IEF
SPY,1.000000,0.895036,0.980181,0.607583,0.550568,0.692868,-0.047666,-0.358972,0.978591,-0.113257,...,0.121639,0.932949,0.919193,0.951714,0.968568,0.976177,0.962110,0.599040,0.866813,0.608727
IWM,0.895036,1.000000,0.815367,0.386384,0.426366,0.373841,-0.026246,-0.129748,0.800497,0.288120,...,0.386162,0.749673,0.952558,0.964325,0.812804,0.924536,0.824791,0.227710,0.606097,0.227099
QQQ,0.980181,0.815367,1.000000,0.631654,0.539391,0.762375,-0.094243,-0.432096,0.997878,-0.260840,...,0.041141,0.915038,0.853778,0.880048,0.975562,0.924489,0.983416,0.697075,0.894190,0.702148
GDX,0.607583,0.386384,0.631654,1.000000,0.941442,0.893559,0.571450,-0.514198,0.656093,-0.313485,...,-0.251183,0.708550,0.453767,0.505850,0.595329,0.522397,0.573109,0.796715,0.830596,0.788048
GDXJ,0.550568,0.426366,0.539391,0.941442,1.000000,0.739133,0.663391,-0.515078,0.560256,-0.152734,...,-0.203427,0.644176,0.486679,0.515289,0.497276,0.486313,0.473020,0.622935,0.718635,0.624363
GLD,0.692868,0.373841,0.762375,0.893559,0.739133,1.000000,0.312693,-0.521111,0.786348,-0.547957,...,-0.305152,0.771959,0.473180,0.521821,0.723062,0.595216,0.712157,0.935112,0.914299,0.922029
SLV,-0.047666,-0.026246,-0.094243,0.571450,0.663391,0.312693,1.000000,-0.049815,-0.066053,0.155420,...,-0.066880,0.069898,-0.030511,0.009810,-0.118571,-0.054158,-0.154222,0.157598,0.185505,0.138717
USO,-0.358972,-0.129748,-0.432096,-0.514198,-0.515078,-0.521111,-0.049815,1.000000,-0.435273,0.691060,...,0.548864,-0.464076,-0.299403,-0.247951,-0.374127,-0.251472,-0.363798,-0.608716,-0.529386,-0.642149
XLK,0.978591,0.800497,0.997878,0.656093,0.560256,0.786348,-0.066053,-0.435273,1.000000,-0.277798,...,0.019466,0.928405,0.845170,0.875224,0.970995,0.923926,0.974678,0.721900,0.913002,0.727965
XLE,-0.113257,0.288120,-0.260840,-0.313485,-0.152734,-0.547957,0.155420,0.691060,-0.277798,1.000000,...,0.704176,-0.276617,0.107741,0.123640,-0.226235,0.003630,-0.204958,-0.710447,-0.429528,-0.735325


In [50]:
ticker_df['']

In [56]:
corr_matrix['SPY']['GLD'] = series_corr(ticker_df['SPY'], ticker_df['GLD'])

In [57]:
corr_matrix['SPY']['GLD']

0.7021314401497865